<a href="https://colab.research.google.com/github/aCStandke/ReinforcementLearning/blob/main/SecondStockEnivornment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# !pip uninstall -y tensorflow
# !tensorflow-gpu==1.14
!pip install stable-baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np
import json
import datetime as dt
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import collections
import datetime

In [15]:
# Stock Environment Parameters
MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 5000
MAX_OPEN_POSITIONS = 5
MAX_STEPS = 20000
TRADING_DAYS = 5
INITIAL_ACCOUNT_BALANCE = 10000

In [39]:
# Stock/ETF Trading Enviornment
class StockTradingEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, data):
        super(StockTradingEnv, self).__init__()

        self.data = data
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)

        # Actions of the format Buy x%, Sell x%, Hold, etc.
        self.action_space = spaces.Box(
            low=np.array([0, 0]), high=np.array([3, 1]))

        # Prices contains the OHCL values for the last five prices
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(31,))

    def _next_observation(self):
        # Get the stock data points for the last 5 days and scale to between 0-1
        frame = np.array([
                    data.high[self.current_step: self.current_step +TRADING_DAYS],
                    data.low[self.current_step: self.current_step +TRADING_DAYS],
                    data.close[self.current_step: self.current_step +TRADING_DAYS],
                    data.open[self.current_step: self.current_step +TRADING_DAYS],
                    data.volume[self.current_step: self.current_step +TRADING_DAYS]
                    ])

        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]])

        return obs

    def _take_action(self, action):
        # Set the current price to a random price within the time step
        current_price = random.uniform(
            self.data.open[self.current_step], self.data.close[self.current_step])

        action_type = action[0]
        amount = action[1]

        if action_type < 1:
            # Buy amount % of balance in shares
            total_possible = int(self.balance / (1+current_price))
            shares_bought = int(total_possible * amount)
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price

            self.balance -= additional_cost
            self.cost_basis = (
                prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought

        elif action_type < 2:
            # Sell amount % of shares held
            shares_sold = int(self.shares_held * amount)
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price

        self.net_worth = self.balance + self.shares_held * current_price

        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth

        if self.shares_held == 0:
            self.cost_basis = 0

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        if self.current_step > len(self.data.open[:]) - 6:
            self.current_step = 0

        delay_modifier = (self.current_step / MAX_STEPS)

        reward = self.balance * delay_modifier
        done = self.net_worth <= 0

        obs = self._next_observation()

        return obs, reward, done, {}

    def reset(self):
        # Reset the state of the environment to an initial state
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Set the current step to a random point within the data frame
        self.current_step = random.randint(
            0, len(self.data.open[:]) - 6)

        return self._next_observation()

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE

        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(
            f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
        print(
            f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
        print(
            f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')


In [17]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/StockMarketData/archive/Data/ETFs/spy.us.txt')
df = df.sort_values('Date')
data=df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

In [18]:
dt = data['Date'].array
rh = (data['High'].values-data['Open'].values)/data['Open'].values
rl = (data['Low'].values-data['Open'].values)/data['Open'].values
rc = (data['Close'].values-data['Open'].values)/data['Open'].values
ro = (data['Open'].values-data['Close'].values)/data['Close'].values
vol = data['Volume'].values

In [19]:
Data = collections.namedtuple('Data', field_names=['date','high', 'low', 'close', 'open', 'volume'])

In [20]:
data=Data(date=dt,high=rh, low=rl, close=rc, open=ro, volume=vol)

In [ ]:
# frame = np.array([
#     data.date[0:5], 
#     data.high[0:5],
#     data.low[0:5],
#     data.close[0:5],
#     data.open[0:5],
#     data.volume[0:5]
# ])

In [ ]:
# obs = np.append(frame,[[
#       INITIAL_ACCOUNT_BALANCE / MAX_ACCOUNT_BALANCE,
#       INITIAL_ACCOUNT_BALANCE / MAX_ACCOUNT_BALANCE,
#       0 / MAX_NUM_SHARES,
#       0 / MAX_SHARE_PRICE,
#       0 / MAX_NUM_SHARES,
#       0 / (MAX_NUM_SHARES * MAX_SHARE_PRICE)
#         ]])

In [38]:
random.uniform(data.open[20], data.close[20])

0.00022270560485814568

In [40]:
# The algorithms require a vectorized environment to run
env = StockTradingEnv(data)
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [41]:
model.learn(total_timesteps=20000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


-----------------------------
| time/              |      |
|    fps             | 1355 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


-------------------------------------------
| time/                   |               |
|    fps                  | 859           |
|    iterations           | 2             |
|    time_elapsed         | 4             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00092552445 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.84         |
|    explained_variance   | 5.36e-07      |
|    learning_rate        | 0.0003        |
|    loss                 | 1.52e+08      |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.00108      |
|    std                  | 1             |
|    value_loss           | 3.1e+08       |
-------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


...

------------------------------------------
| time/                   |              |
|    fps                  | 800          |
|    iterations           | 5            |
|    time_elapsed         | 12           |
|    total_timesteps      | 10240        |
| train/                  |              |
|    approx_kl            | 0.0020914797 |
|    clip_fraction        | 0.000293     |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.85        |
|    explained_variance   | -7.27e-06    |
|    learning_rate        | 0.0003       |
|    loss                 | 2.19e+07     |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.000975    |
|    std                  | 1            |
|    value_loss           | 4.21e+07     |
------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


------------------------------------------
| time/                   |              |
|    fps                  | 799          |
|    iterations           | 6            |
|    time_elapsed         | 15           |
|    total_timesteps      | 12288        |
| train/                  |              |
|    approx_kl            | 0.0030655852 |
|    clip_fraction        | 0.00508      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.85        |
|    explained_variance   | -5.96e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 1.24e+07     |
|    n_updates            | 50           |
|    policy_gradient_loss | -0.00147     |
|    std                  | 1            |
|    value_loss           | 2.37e+07     |
------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


KeyboardInterrupt: ignored

In [ ]:
# obs = env.reset()
# for i in range(2000):
#   action, _states = model.predict(obs)
#   obs, rewards, done, info = env.step(action)
#   env.render()